<a href="https://colab.research.google.com/github/gocgodman/M2M/blob/main/Transkun_Colab_Notebook_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 셀 1: 필요한 패키지 설치 (한 번만 실행)
!pip install -q yt-dlp gdown transkun gradio pretty_midi librosa numpy soundfile pyfluidsynth
!apt-get update -qq
!apt-get install -y -qq ffmpeg fluidsynth p7zip-full || true

In [ ]:

# 셀 2: Drive 마운트 및 기본 경로 설정
from google.colab import drive
drive.mount('/content/drive')

import os, uuid
WORK_ROOT = "/content/ytd_pipeline_" + uuid.uuid4().hex
os.makedirs(WORK_ROOT, exist_ok=True)

DRIVE_SF2_DIR = "/content/drive/MyDrive/sf2_library"
DRIVE_RESULTS_DIR = "/content/drive/MyDrive/ytd_pipeline_results"
os.makedirs(DRIVE_SF2_DIR, exist_ok=True)
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)

print("WORK_ROOT:", WORK_ROOT)
print("DRIVE_SF2_DIR:", DRIVE_SF2_DIR)
print("DRIVE_RESULTS_DIR:", DRIVE_RESULTS_DIR)

In [ ]:

# 셀 3: Drive 폴더 ID로 SF2 수집 및 압축 해제
import os, glob, zipfile, tarfile, subprocess, shutil
import gdown

# 필요하면 여기 Drive 폴더 ID를 바꾸세요
DRIVE_FOLDER_ID = "1JkTMvPwM_XURqG2114n4Qj0rR83WEucL"

OUT_DIR = "/content/sf2_from_drive"
os.makedirs(OUT_DIR, exist_ok=True)

if DRIVE_FOLDER_ID:
    try:
        gdown.download_folder(id=DRIVE_FOLDER_ID, output=OUT_DIR, quiet=False, use_cookies=False)
    except Exception as e:
        print("gdown.download_folder 실패:", e)

def try_extract_archive(path, dest):
    path_lower = path.lower()
    os.makedirs(dest, exist_ok=True)
    try:
        if path_lower.endswith(".zip"):
            with zipfile.ZipFile(path, 'r') as zf:
                zf.extractall(dest); return True
        if path_lower.endswith((".tar.gz", ".tgz", ".tar")):
            with tarfile.open(path, 'r:*') as tf:
                tf.extractall(dest); return True
        if path_lower.endswith(".7z"):
            cmd = ['7z', 'x', '-y', '-o' + dest, path]
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE); return True
    except Exception as e:
        print("압축 해제 실패:", e)
    return False

# 압축 해제 및 .sf2 수집
for root, dirs, files in os.walk(OUT_DIR):
    for fn in files:
        full = os.path.join(root, fn)
        if fn.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar", ".7z")):
            extract_dest = os.path.join(root, fn + "_extracted")
            try_extract_archive(full, extract_dest)

sf2_files = []
for root, dirs, files in os.walk(OUT_DIR):
    for fn in files:
        if fn.lower().endswith(".sf2"):
            sf2_files.append(os.path.join(root, fn))

# Drive로 복사
copied = []
for p in sf2_files:
    dest = os.path.join(DRIVE_SF2_DIR, os.path.basename(p))
    try:
        shutil.copy(p, dest)
        copied.append(dest)
    except Exception as e:
        print("복사 실패:", p, e)

print("발견된 .sf2 수:", len(sf2_files))
for p in copied:
    print(" -", p)

In [ ]:

# 통합 함수 정의 셀 — 이 셀 하나로 파이프라인의 모든 핵심 함수(다운로드, 전사, 페달 검출, MIDI 삽입,
# 렌더링, 체크포인트, 재생목록 제너레이터)를 정의합니다.
# 이전에 실행한 다른 셀들과 충돌하지 않도록 함수/상수 이름을 동일하게 유지했습니다.
# 이 셀을 실행한 뒤에는 UI 셀(Gradio)이나 실행 셀에서 바로 호출할 수 있습니다.

# ---------------- 기본 라이브러리 및 상수 ----------------
import os
import sys
import time
import uuid
import json
import glob
import shutil
import zipfile
import tarfile
import subprocess
from pathlib import Path

import numpy as np
import librosa
import pretty_midi
from scipy.ndimage import uniform_filter1d, binary_closing

# 결과 저장 폴더(필요시 변경)
DRIVE_RESULTS_DIR = "/content/drive/MyDrive/ytd_pipeline_results"
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)

# 임시 작업 루트
WORK_ROOT = "/content/ytd_pipeline_work"
os.makedirs(WORK_ROOT, exist_ok=True)

# 상태 파일 경로
STATE_FILE = os.path.join(DRIVE_RESULTS_DIR, "pipeline_state.json")

# ---------------- 유틸리티: 명령 실행 ----------------
def run_cmd(cmd, check=False):
    """서브프로세스 실행 유틸. check=True면 예외 발생."""
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if check and proc.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\nSTDOUT:{proc.stdout}\nSTDERR:{proc.stderr}")
    return proc.returncode, proc.stdout, proc.stderr

# ---------------- yt-dlp 헬퍼 (단일/재생목록 확장/재생목록 전체 다운로드) ----------------
def expand_playlist_to_video_urls(playlist_url):
    """
    yt-dlp --flat-playlist -J 으로 재생목록 항목의 video id 목록을 추출하여
    https://www.youtube.com/watch?v=... 형식의 URL 리스트로 반환합니다.
    """
    cmd = ["yt-dlp", "--flat-playlist", "-J", playlist_url]
    code, out, err = run_cmd(cmd)
    if code != 0:
        raise RuntimeError(f"yt-dlp playlist expand failed: {err}")
    j = json.loads(out)
    entries = j.get("entries", [])
    urls = []
    for e in entries:
        vid = e.get("id")
        if vid:
            urls.append(f"https://www.youtube.com/watch?v={vid}")
    return urls

def download_youtube_audio_single(url, outdir, fmt="mp3", audio_bitrate="192K"):
    """
    단일 유튜브 영상에서 오디오 추출(mp3) 후 파일 경로 반환.
    outdir에 저장되며, 실패 시 예외 발생.
    """
    os.makedirs(outdir, exist_ok=True)
    out_template = os.path.join(outdir, "%(playlist_index)s-%(title)s.%(ext)s")
    cmd = [
        "yt-dlp",
        "-x", "--audio-format", fmt, "--audio-quality", audio_bitrate,
        "-o", out_template,
        url
    ]
    code, out, err = run_cmd(cmd)
    # yt-dlp가 non-zero를 반환해도 파일이 생성될 수 있으므로 파일 존재 여부로 판단
    files = sorted(glob.glob(os.path.join(outdir, f"*.{fmt}")))
    if not files:
        raise RuntimeError(f"No audio file produced by yt-dlp for {url}. yt-dlp stderr: {err}")
    return files[-1]

def download_youtube_playlist_audio(playlist_url, outdir, fmt="mp3", audio_bitrate="192K"):
    """
    재생목록 전체를 mp3로 추출. outdir에 저장된 파일 경로 리스트 반환.
    """
    os.makedirs(outdir, exist_ok=True)
    out_template = os.path.join(outdir, "%(playlist_index)s-%(title)s.%(ext)s")
    cmd = [
        "yt-dlp",
        "-x", "--audio-format", fmt, "--audio-quality", audio_bitrate,
        "-o", out_template,
        playlist_url
    ]
    code, out, err = run_cmd(cmd)
    files = sorted(glob.glob(os.path.join(outdir, f"*.{fmt}")))
    return files

# ---------------- 전사(Transcription) 래퍼 ----------------
def remove_extension(filepath):
    return ".".join(os.path.basename(filepath).split('.')[:-1])

def transcribe_file(input_path, outfolder='.'):
    """
    transkun을 호출해 input_path를 MIDI로 변환. outfolder에 mid 파일 생성 후 경로 반환.
    CUDA 옵션을 우선 시도하고 실패하면 CPU로 재시도합니다.
    """
    base = remove_extension(input_path)
    out_mid = os.path.join(outfolder, base + ".mid")
    cmd = ['transkun', input_path, out_mid, '--device', 'cuda']
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except Exception:
        cmd = ['transkun', input_path, out_mid]
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return out_mid

# ---------------- 페달 검출 알고리즘 ----------------
def compute_spectral_flux(y, sr, hop_length, n_fft=2048):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_norm = S / (np.sum(S, axis=0, keepdims=True) + 1e-8)
    flux = np.sqrt(np.sum(np.diff(S_norm, axis=1).clip(min=0)**2, axis=0))
    return np.concatenate(([0.0], flux))

def detect_pedal_advanced(audio_path,
                          sr=22050,
                          hop_length=512,
                          energy_smooth=0.5,
                          low_freq_cut=500,
                          low_energy_weight=0.6,
                          flux_weight=0.4,
                          on_threshold=1.0,
                          off_threshold=0.7,
                          min_event_len=0.08,
                          merge_gap=0.08,
                          closing_size=3):
    """
    오디오 파일에서 페달(서스테인) 구간을 검출하여 (start, end) 초 단위 튜플 리스트 반환.
    파라미터로 민감도와 스무딩을 조절할 수 있습니다.
    """
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    frame_energy = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop_length)[0]
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)
    low_idx = np.where(freqs <= low_freq_cut)[0]
    low_energy = S[low_idx, :].sum(axis=0) if len(low_idx) > 0 else np.zeros_like(frame_energy)
    e = frame_energy / (frame_energy.max() + 1e-8)
    le = low_energy / (low_energy.max() + 1e-8) if low_energy.max() > 0 else low_energy
    flux = compute_spectral_flux(y, sr, hop_length)
    flux = flux / (flux.max() + 1e-8)
    combined = low_energy_weight * le + (1.0 - flux_weight) * (1.0 - flux) + 0.4 * e
    window = int(max(1, energy_smooth * (sr / hop_length)))
    combined_smooth = uniform_filter1d(combined, size=window)
    mu = np.mean(combined_smooth); sigma = np.std(combined_smooth) + 1e-8
    on_thr = mu + on_threshold * sigma; off_thr = mu + off_threshold * sigma
    on_mask = np.zeros_like(combined_smooth, dtype=bool)
    state = False
    for i, val in enumerate(combined_smooth):
        if not state and val >= on_thr:
            state = True; on_mask[i] = True
        elif state:
            on_mask[i] = True
            if val < off_thr:
                state = False
    if closing_size > 1:
        on_mask = binary_closing(on_mask, structure=np.ones(closing_size))
    times = librosa.frames_to_time(np.arange(len(on_mask)), sr=sr, hop_length=hop_length)
    events = []; prev=False; start_t=None
    for t,m in zip(times, on_mask):
        if m and not prev: start_t=t
        if (not m) and prev and start_t is not None:
            events.append((start_t,t)); start_t=None
        prev=m
    if prev and start_t is not None: events.append((start_t,times[-1]))
    filtered=[]
    for (s,e) in events:
        if (e-s) >= min_event_len:
            if filtered and s - filtered[-1][1] <= merge_gap:
                filtered[-1] = (filtered[-1][0], e)
            else:
                filtered.append((s,e))
    return filtered

# ---------------- MIDI에 페달 CC 삽입 ----------------
def insert_pedal_cc_into_midi(midi_in_path, midi_out_path, pedal_events, piano_program=0):
    """
    pretty_midi를 사용해 pedal_events를 컨트롤 체인지(64)로 삽입하고 저장.
    """
    pm = pretty_midi.PrettyMIDI(midi_in_path)
    for inst in pm.instruments:
        inst.program = piano_program
    target_inst = pm.instruments[0] if pm.instruments else pretty_midi.Instrument(program=piano_program)
    if not pm.instruments:
        pm.instruments.append(target_inst)
    for (s,e) in pedal_events:
        on_time = max(0.0, s - 0.02); off_time = e + 0.02
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=127, time=on_time))
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=0, time=off_time))
    for inst in pm.instruments:
        inst.control_changes.sort(key=lambda cc: cc.time)
    pm.write(midi_out_path)

# ---------------- fluidsynth 렌더링 ----------------
def render_midi_to_wav(midi_path, wav_out_path, sf2_path, sample_rate=44100):
    """
    fluidsynth를 사용해 MIDI를 WAV로 렌더링. sf2_path가 없거나 존재하지 않으면 None 반환.
    """
    if sf2_path is None or not os.path.exists(sf2_path):
        return None
    try:
        cmd = ['fluidsynth', '-ni', sf2_path, midi_path, '-F', wav_out_path, '-r', str(sample_rate)]
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return wav_out_path
    except Exception as e:
        # 렌더 실패 시 None 반환
        return None

# ---------------- 체크포인트(상태) 유틸리티 ----------------
def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"processed": [], "failed": [], "items": {}, "last_update": None}

def save_state(state):
    state["last_update"] = time.time()
    with open(STATE_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=2)

# ---------------- 항목 단위 처리 래퍼 ----------------
def process_item_from_url_or_path(item, sf2_path=None, tmp_dir=None):
    """
    item: YouTube URL 또는 로컬 오디오 파일 경로
    sf2_path: 렌더링에 사용할 SF2 경로(문자열) 또는 None
    tmp_dir: 임시 작업 디렉터리(지정하지 않으면 WORK_ROOT 기반 생성)
    반환: dict(status, mp3, midi, wav, pedals or error)
    """
    if tmp_dir is None:
        tmp_dir = os.path.join(WORK_ROOT, "tmp_" + uuid.uuid4().hex)
    os.makedirs(tmp_dir, exist_ok=True)
    try:
        # 1) 오디오 확보
        if isinstance(item, str) and item.startswith("http"):
            mp3 = download_youtube_audio_single(item, tmp_dir, fmt="mp3")
        else:
            mp3 = item

        # 2) 전사
        mid = transcribe_file(mp3, outfolder=tmp_dir)

        # 3) 페달 검출 및 삽입
        pedals = detect_pedal_advanced(mp3)
        mid_pedal = os.path.join(tmp_dir, os.path.splitext(os.path.basename(mid))[0] + "_pedal.mid")
        insert_pedal_cc_into_midi(mid, mid_pedal, pedals, piano_program=0)

        # 4) Drive에 저장 (mp3, midi)
        saved_mp3 = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(mp3)); shutil.copy(mp3, saved_mp3)
        saved_midi = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(mid_pedal)); shutil.copy(mid_pedal, saved_midi)

        # 5) 선택 SF2로 렌더링
        saved_wav = None
        if sf2_path:
            wav_out = os.path.join(tmp_dir, os.path.splitext(os.path.basename(mp3))[0] + "_render.wav")
            rendered = render_midi_to_wav(mid_pedal, wav_out, sf2_path)
            if rendered:
                saved_wav = os.path.join(DRIVE_RESULTS_DIR, os.path.basename(wav_out)); shutil.copy(wav_out, saved_wav)

        return {"status":"ok", "mp3": saved_mp3, "midi": saved_midi, "wav": saved_wav, "pedals": pedals}
    except Exception as e:
        return {"status":"error", "error": str(e)}

# ---------------- 파일 업로드 기반 처리 함수 (기존 process_files와 동일한 시그니처) ----------------
def merge_sustain_notes(midi_path, midi_out_path, tolerance=0.2):
    """
    같은 음(pitch)이 짧게 끊겨 여러 개로 인식된 경우 tolerance 이하 간격이면 하나로 병합.
    """
    pm = pretty_midi.PrettyMIDI(midi_path)
    for inst in pm.instruments:
        merged_notes = []
        if not inst.notes:
            continue
        inst.notes.sort(key=lambda n: (n.start, n.end))
        current = inst.notes[0]
        for nxt in inst.notes[1:]:
            if nxt.pitch == current.pitch and nxt.start - current.end <= tolerance:
                current.end = max(current.end, nxt.end)
            else:
                merged_notes.append(current)
                current = nxt
        merged_notes.append(current)
        inst.notes = merged_notes
    pm.write(midi_out_path)


def process_files(gr_files,
                  sf2_choice,
                  uploaded_sf2,
                  sustain_tolerance=0.2,
                  pedal_energy_smooth=0.5,
                  pedal_low_freq_cut=500,
                  pedal_low_energy_weight=0.6,
                  pedal_flux_weight=0.4,
                  pedal_on_threshold=1.0,
                  pedal_off_threshold=0.7,
                  pedal_min_event_len=0.08,
                  pedal_merge_gap=0.08):
    """
    파일 업로드 기반 처리 함수 — 노트 병합 + 페달 삽입 + 렌더링 포함
    """
    chosen_sf2 = None
    if sf2_choice and sf2_choice != "None":
        chosen_sf2 = sf2_choice
    if uploaded_sf2:
        if isinstance(uploaded_sf2, list) and len(uploaded_sf2) > 0:
            up = uploaded_sf2[0]
            chosen_sf2 = up['name'] if isinstance(up, dict) and 'name' in up else up
    if not gr_files:
        return None, None

    work_dir = "/tmp/transcribe_" + uuid.uuid4().hex
    os.makedirs(work_dir, exist_ok=True)
    local_paths = []
    for f in gr_files:
        if isinstance(f, dict) and 'name' in f:
            local_paths.append(f['name'])
        else:
            local_paths.append(f)

    midi_outs = []
    for audio_path in local_paths:
        try:
            mid = transcribe_file(audio_path, outfolder=work_dir)
        except Exception as e:
            return f"Transcription failed: {e}", None

        # --- 추가: 노트 병합 ---
        merged_mid = os.path.join(work_dir, remove_extension(os.path.basename(mid)) + "_merged.mid")
        merge_sustain_notes(mid, merged_mid, tolerance=sustain_tolerance)

        # --- 페달 검출 및 삽입 ---
        pedals = detect_pedal_advanced(audio_path,
                                       sr=22050,
                                       hop_length=512,
                                       energy_smooth=pedal_energy_smooth,
                                       low_freq_cut=pedal_low_freq_cut,
                                       low_energy_weight=pedal_low_energy_weight,
                                       flux_weight=pedal_flux_weight,
                                       on_threshold=pedal_on_threshold,
                                       off_threshold=pedal_off_threshold,
                                       min_event_len=pedal_min_event_len,
                                       merge_gap=pedal_merge_gap,
                                       closing_size=3)
        out_mid = os.path.join(work_dir, remove_extension(os.path.basename(merged_mid)) + "_pedal.mid")
        insert_pedal_cc_into_midi(merged_mid, out_mid, pedals, piano_program=0)
        midi_outs.append(out_mid)

    # 여러 파일이면 ZIP으로 묶기
    if len(midi_outs) == 1:
        final_midi = midi_outs[0]
    else:
        zip_path = os.path.join(work_dir, "results_with_pedal.zip")
        with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
            for p in midi_outs:
                zf.write(p, arcname=os.path.basename(p))
        final_midi = zip_path

    # 렌더링
    wav_path = None
    if chosen_sf2:
        try:
            first_mid = midi_outs[0]
            wav_out = os.path.join(work_dir, remove_extension(os.path.basename(first_mid)) + ".wav")
            rendered = render_midi_to_wav(first_mid, wav_out, sf2_path=chosen_sf2)
            if rendered:
                wav_path = rendered
        except Exception:
            wav_path = None

    return final_midi, wav_path

# ---------------- 재생목록 제너레이터 (진행도 스트리밍용) ----------------
def playlist_pipeline_generator(playlist_text, sf2_choice_path):
    """
    재생목록(또는 여러 URL) 텍스트를 받아 각 항목을 처리하면서 진행 로그를 yield 합니다.
    yield 형식: (로그문자열, 중간결과 dict 또는 None)
    """
    # 의존성 확인
    missing = []
    for name in ("expand_playlist_to_video_urls", "process_item_from_url_or_path", "load_state", "save_state"):
        if name not in globals():
            missing.append(name)
    if missing:
        yield f"필수 함수가 정의되어 있지 않습니다: {', '.join(missing)}", None
        return

    lines = [ln.strip() for ln in str(playlist_text).splitlines() if ln.strip()]
    items = []
    for ln in lines:
        if ln.startswith("http") and ("playlist" in ln or "list=" in ln):
            yield f"재생목록 확장 중: {ln}", None
            try:
                urls = expand_playlist_to_video_urls(ln)
                if not urls:
                    yield f"재생목록에서 URL을 찾지 못했습니다: {ln}", None
                else:
                    items.extend(urls)
                    yield f"재생목록에서 {len(urls)}개 항목 발견", None
            except Exception as e:
                yield f"재생목록 확장 실패: {e}", None
        else:
            items.append(ln)

    total = len(items)
    if total == 0:
        yield "처리할 항목이 없습니다.", None
        return

    try:
        state = load_state()
    except Exception as e:
        yield f"상태 파일 로드 실패: {e}", None
        return

    yield f"총 항목: {total}. 이미 처리된 항목: {len(state.get('processed', []))}. 처리 시작합니다...", None

    for idx, item in enumerate(items, start=1):
        if item in state.get("processed", []):
            yield f"[{idx}/{total}] 이미 처리됨: {item}", None
            continue

        yield f"[{idx}/{total}] 다운로드/전사 시작: {item}", None
        try:
            res = process_item_from_url_or_path(item, sf2_path=sf2_choice_path)
        except Exception as e:
            res = {"status": "error", "error": str(e)}

        if res.get("status") == "ok":
            state.setdefault("processed", []).append(item)
            state.setdefault("items", {})[item] = {
                "mp3": res.get("mp3"),
                "midi": res.get("midi"),
                "wav": res.get("wav"),
                "time": time.time()
            }
            try:
                save_state(state)
            except Exception as e:
                yield f"[{idx}/{total}] 완료했으나 상태 저장 실패: {e}", res
                continue
            yield f"[{idx}/{total}] 완료: {item} → MIDI: {os.path.basename(res.get('midi') or '')} WAV: {os.path.basename(res.get('wav') or '') or '없음'}", res
        else:
            state.setdefault("failed", []).append({"item": item, "error": res.get("error")})
            try:
                save_state(state)
            except Exception as e:
                yield f"[{idx}/{total}] 실패: {item} 에러 저장 실패: {e}", None
                continue
            yield f"[{idx}/{total}] 실패: {item} 에러: {res.get('error')}", None

    yield "모든 항목 처리 완료(또는 건너뛰기 완료). Drive의 pipeline_state.json을 확인하세요.", None

# ---------------- 완료 메시지 ----------------
print("통합 함수 정의 완료: yt-dlp 헬퍼, 전사, 페달 검출, MIDI 삽입, 렌더링, 체크포인트, 재생목록 제너레이터가 준비되었습니다.")

In [ ]:

# 완전한 통합 UI 셀 — 업로드 탭 + 재생목록 탭, SF2 선택 포함 (단일 셀로 실행)
# 이 셀은 이전에 정의된 함수들(process_files, playlist_pipeline_generator 등)을 사용합니다.
# 만약 해당 함수들이 아직 정의되지 않았다면 먼저 관련 셀들을 실행하세요.

import gradio as gr
import os, traceback

# sf2_files가 이미 존재하면 사용, 아니면 빈 리스트
try:
    sf2_choices = ["None"] + sf2_files
except Exception:
    sf2_choices = ["None"]

def get_sf2_choice_value(choice, uploaded_sf2):
    """
    UI에서 선택된 SF2 경로를 결정:
    - uploaded_sf2가 있으면 업로드된 파일(첫번째)을 우선 사용
    - 아니면 드롭다운 선택(choice)를 사용
    """
    chosen = None
    try:
        if uploaded_sf2 and isinstance(uploaded_sf2, list) and len(uploaded_sf2) > 0:
            up = uploaded_sf2[0]
            if isinstance(up, dict) and 'name' in up:
                chosen = up['name']
            else:
                chosen = up
    except Exception:
        chosen = None
    if not chosen and choice and choice != "None":
        chosen = choice
    return chosen

# 제너레이터 래퍼: 업로드 우선 SF2 결정 후 내부 제너레이터에 위임
def playlist_wrapper_with_upload(playlist_text, sf2_choice_val, uploaded_sf2_val, sf2_direct_input):
    try:
        chosen = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val)
        if sf2_direct_input:
            chosen = sf2_direct_input
        # playlist_pipeline_generator는 제너레이터여야 함
        yield from playlist_pipeline_generator(playlist_text, chosen)
    except Exception as e:
        # 에러 발생 시 사용자에게 즉시 스트리밍 형태로 알림
        yield f"재생목록 처리 중 예외 발생: {e}", None

# 업로드 처리 콜백(동기 함수)
def on_run_upload(gr_files, sf2_choice_val, uploaded_sf2_val,
                  sustain_tolerance, pedal_energy_smooth, pedal_low_freq_cut,
                  pedal_low_energy_weight, pedal_flux_weight,
                  pedal_on_threshold, pedal_off_threshold,
                  pedal_min_event_len, pedal_merge_gap):
    try:
        chosen_sf2 = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val)
        status = f"업로드 처리 시작. SF2: {os.path.basename(chosen_sf2) if chosen_sf2 else '없음'}"
        final_midi, wav_path = process_files(
            gr_files,
            sf2_choice=sf2_choice_val,
            uploaded_sf2=uploaded_sf2_val,
            sustain_tolerance=sustain_tolerance,
            pedal_energy_smooth=pedal_energy_smooth,
            pedal_low_freq_cut=pedal_low_freq_cut,
            pedal_low_energy_weight=pedal_low_energy_weight,
            pedal_flux_weight=pedal_flux_weight,
            pedal_on_threshold=pedal_on_threshold,
            pedal_off_threshold=pedal_off_threshold,
            pedal_min_event_len=pedal_min_event_len,
            pedal_merge_gap=pedal_merge_gap
        )
        return status, final_midi, wav_path
    except Exception as e:
        tb = traceback.format_exc()
        return f"업로드 처리 중 오류: {e}\n{tb}", None, None

# Blocks UI 생성 및 실행
with gr.Blocks() as combined_demo:
    gr.Markdown("## 통합: 파일 업로드 + 재생목록 배치 처리 (공통 SF2 선택)")
    with gr.Row():
        with gr.Column(scale=1):
            sf2_dropdown = gr.Dropdown(choices=sf2_choices, value=sf2_choices[0], label="Drive에서 선택할 SF2")
            sf2_upload = gr.Files(file_types=[".sf2"], label="또는 SF2 업로드 (선택)")
            gr.Markdown("**설명:** 업로드된 SF2가 있으면 업로드된 파일을 우선 사용합니다. 드롭다운에서 Drive에 복사된 SF2를 선택할 수도 있습니다.")
        with gr.Column(scale=2):
            status_box = gr.Textbox(label="전역 상태", interactive=False)
            status_box.value = f"결과 폴더: {DRIVE_RESULTS_DIR if 'DRIVE_RESULTS_DIR' in globals() else '설정 필요'}"

    with gr.Tabs():
        with gr.TabItem("파일 업로드 처리"):
            upload_files = gr.Files(file_types=[".wav", ".mp3"], label="오디오 파일 업로드")
            sustain = gr.Slider(0.05, 0.6, value=0.2, step=0.01, label="Sustain tolerance (note merge, sec)")
            pedal_smooth = gr.Slider(0.1, 1.0, value=0.5, step=0.05, label="Pedal energy smoothing (sec)")
            pedal_lowcut = gr.Slider(200, 2000, value=500, step=50, label="Pedal low-frequency cutoff (Hz)")
            pedal_low_w = gr.Slider(0.0, 1.0, value=0.6, step=0.05, label="Low-energy weight (pedal)")
            pedal_flux_w = gr.Slider(0.0, 1.0, value=0.4, step=0.05, label="Flux weight (pedal)")
            pedal_on = gr.Slider(0.2, 2.0, value=1.0, step=0.1, label="Pedal ON threshold (sigma multiplier)")
            pedal_off = gr.Slider(0.0, 1.5, value=0.7, step=0.05, label="Pedal OFF threshold (sigma multiplier)")
            pedal_min = gr.Slider(0.02, 0.5, value=0.08, step=0.01, label="Min pedal event length (sec)")
            pedal_merge = gr.Slider(0.01, 0.3, value=0.08, step=0.01, label="Merge gap for pedal events (sec)")
            run_upload = gr.Button("업로드 파일 처리 시작")
            upload_result_file = gr.File(label="Download MIDI or ZIP")
            upload_preview = gr.Audio(label="Preview (WAV) - rendered with chosen SF2", type="filepath")

            run_upload.click(
                fn=on_run_upload,
                inputs=[upload_files, sf2_dropdown, sf2_upload,
                        sustain, pedal_smooth, pedal_lowcut, pedal_low_w, pedal_flux_w,
                        pedal_on, pedal_off, pedal_min, pedal_merge],
                outputs=[status_box, upload_result_file, upload_preview]
            )

        with gr.TabItem("재생목록 배치 처리"):
            playlist_input = gr.Textbox(lines=6, label="재생목록 URL 또는 영상 URL들 (줄바꿈으로 여러개 입력)")
            playlist_sf2_input = gr.Textbox(label="(선택) SF2 경로를 직접 입력하거나 위 드롭다운/업로드 사용")
            run_playlist_btn = gr.Button("재생목록 처리 시작")
            playlist_log = gr.Textbox(label="진행 로그")
            playlist_last = gr.JSON(label="마지막 항목 결과")

            # 제너레이터 래퍼를 직접 바인딩 (Gradio는 제너레이터 함수를 스트리밍으로 처리)
            run_playlist_btn.click(
                fn=playlist_wrapper_with_upload,
                inputs=[playlist_input, sf2_dropdown, sf2_upload, playlist_sf2_input],
                outputs=[playlist_log, playlist_last]
            )

    gr.Markdown("**사용법:** 상단에서 SF2를 드롭다운으로 선택하거나 SF2 파일을 업로드하세요. 업로드 탭은 로컬 파일을 처리하고, 재생목록 탭은 유튜브 재생목록/URL을 배치 처리합니다. 처리 결과는 Drive의 ytd_pipeline_results에 저장됩니다.")

# 실행 (Colab에서는 share=False, debug=True 권장)
combined_demo.launch(share=False, debug=True)